# Secure Hash Standard: Binary Word Operations — Problem 1

## Introduction

This notebook is for the comp-theory module problems where I will document my progress .  
Problem 1 focuses on implementing a set of 32-bit word operations used in the **Secure Hash Standard (SHA-256)**.

These operations include logical functions such as `Ch`, `Maj`, and `Parity`, along with the rotation/shift functions  
`Σ0`, `Σ1`, `σ0`, and `σ1`.

All functions will be implemented in Python using **NumPy**, ensuring that all values behave as unsigned 32-bit integers  
(`np.uint32`). Each function will include:

- a clear docstring  
- a Markdown explanation  
- correct 32-bit behaviour  
- test examples verifying correctness  




## Background

The Secure Hash Standard (NIST FIPS 180-4) defines a series of operations on 32-bit words used by the SHA-256  
compression function. These operations must behave like 32-bit unsigned integers, including wraparound  
behaviour.

### Definitions (FIPS 180-4, Section 4.1.2)

Logical functions:
- **Parity(x, y, z)** = x XOR y XOR z  
- **Ch(x, y, z)** = (x AND y) XOR ((NOT x) AND z)  
- **Maj(x, y, z)** = (x AND y) XOR (x AND z) XOR (y AND z)

Big sigma functions:
- **Σ0(x)** = ROTR²(x) XOR ROTR¹³(x) XOR ROTR²²(x)  
- **Σ1(x)** = ROTR⁶(x) XOR ROTR¹¹(x) XOR ROTR²⁵(x)

Small sigma functions:
- **σ0(x)** = ROTR⁷(x) XOR ROTR¹⁸(x) XOR SHR³(x)  
- **σ1(x)** = ROTR¹⁷(x) XOR ROTR¹⁹(x) XOR SHR¹⁰(x)

Before implementing these, we create helper functions for rotation and logical right shift.


In [24]:
import numpy as np


### Helper Functions: ROTR and SHR

SHA-256 relies on two low-level bit operations:

- **Rotate Right (ROTR)** — circular rotation of x by n bits  
- **Logical Right Shift (SHR)** — shift right, filling with zeroes  

These helper functions ensure correct 32-bit wraparound behaviour.


In [25]:
def rotr(x: np.uint32, n: int) -> np.uint32:
    """
    Rotate Right (ROTR) for 32-bit words.

    Parameters
    ----------
    x : np.uint32
        The 32-bit input word.
    n : int
        Number of bits to rotate.

    Returns
    -------
    np.uint32
        The rotated 32-bit result.
    """
    x = np.uint32(x)
    return np.uint32((x >> n) | (x << (32 - n)))


def shr(x: np.uint32, n: int) -> np.uint32:
    """
    Logical Right Shift (SHR) for 32-bit words.

    Parameters
    ----------
    x : np.uint32
        The 32-bit input word.
    n : int
        Number of bits to shift.

    Returns
    -------
    np.uint32
        The shifted value (zero-filled).
    """
    return np.uint32(x >> n)


### Logical Functions: Parity, Ch, Maj

These functions operate on three 32-bit words.  
They implement core decision and mixing behaviours used by the SHA-256 compression loop.


In [26]:
def Parity(x, y, z):
    """
    Parity function for SHA-256.
    Computes x XOR y XOR z.
    """
    x, y, z = map(np.uint32, (x, y, z))
    return np.uint32(x ^ y ^ z)


def Ch(x, y, z):
    """
    Choice function.
    For each bit of x: if the bit is 1, choose y; otherwise choose z.
    """
    x, y, z = map(np.uint32, (x, y, z))
    return np.uint32((x & y) ^ (~x & z))


def Maj(x, y, z):
    """
    Majority function.
    For each bit, returns the majority value among x, y, z.
    """
    x, y, z = map(np.uint32, (x, y, z))
    return np.uint32((x & y) ^ (x & z) ^ (y & z))


### SHA-256 Sigma Functions

These operations mix input bits using rotations and shifts.  
They form the core of the SHA-256 message schedule and compression function.


In [27]:
def Sigma0(x):
    """Big Sigma 0: ROTR2 ^ ROTR13 ^ ROTR22"""
    x = np.uint32(x)
    return np.uint32(rotr(x, 2) ^ rotr(x, 13) ^ rotr(x, 22))


def Sigma1(x):
    """Big Sigma 1: ROTR6 ^ ROTR11 ^ ROTR25"""
    x = np.uint32(x)
    return np.uint32(rotr(x, 6) ^ rotr(x, 11) ^ rotr(x, 25))


def sigma0(x):
    """Small sigma 0: ROTR7 ^ ROTR18 ^ SHR3"""
    x = np.uint32(x)
    return np.uint32(rotr(x, 7) ^ rotr(x, 18) ^ shr(x, 3))


def sigma1(x):
    """Small sigma 1: ROTR17 ^ ROTR19 ^ SHR10"""
    x = np.uint32(x)
    return np.uint32(rotr(x, 17) ^ rotr(x, 19) ^ shr(x, 10))


### Testing the Functions

We now apply simple tests to verify correct behaviour of all functions.


In [28]:
x = np.uint32(0x12345678)

print("Sigma0 =", hex(Sigma0(x)))
print("Sigma1 =", hex(Sigma1(x)))
print("sigma0 =", hex(sigma0(x)))
print("sigma1 =", hex(sigma1(x)))

a = np.uint32(0xFFFFFFFF)
b = np.uint32(0x00000000)
c = np.uint32(0xAAAAAAAA)

print("Ch(a,b,c) =", hex(Ch(a,b,c)))
print("Maj(a,a,a) =", hex(Maj(a,a,a)))
print("Parity(x,x,x) =", hex(Parity(x,x,x)))


Sigma0 = 0x66146474
Sigma1 = 0x3561abda
sigma0 = 0xe7fce6ee
sigma1 = 0xa1f78649
Ch(a,b,c) = 0x0
Maj(a,a,a) = 0xffffffff
Parity(x,x,x) = 0x12345678


## Conclusion of problem one

In this problem, all SHA-256 binary word operations required by the Secure Hash Standard  
were implemented and verified. NumPy was used to ensure correct 32-bit behaviour.  
This completes Problem 1 and prepares the foundation for message scheduling and compression  
in later problems.


## Problem 2 — Fractional Parts of Cube Roots

In SHA-256, the 64 round constants (often called **K constants**) are derived from the fractional parts of the cube roots of the first 64 prime numbers (NIST FIPS 180-4, p.11).

For each of the first 64 primes \(p_i\):

1. Compute \( \sqrt[3]{p_i} \)
2. Take the fractional part only
3. Multiply by \(2^{32}\)
4. Take the floor to get a 32-bit integer
5. Display in hexadecimal

Below, I generate the primes, compute the constants using NumPy, and verify they match the published values in the Secure Hash Standard.


In [29]:
import numpy as np

def primes(n: int) -> list[int]:
    """
    Generate the first n prime numbers using trial division.

    Parameters
    ----------
    n : int
        Number of primes to generate (n >= 0).

    Returns
    -------
    list[int]
        A list containing the first n primes in ascending order.
    """
    if n < 0:
        raise ValueError("n must be >= 0")
    if n == 0:
        return []

    out: list[int] = []
    candidate = 2

    while len(out) < n:
        is_prime = True
        for p in out:
            if p * p > candidate:
                break
            if candidate % p == 0:
                is_prime = False
                break
        if is_prime:
            out.append(candidate)
        candidate += 1 if candidate == 2 else 2  # after 2, check only odd numbers

    return out


# Quick sanity checks
assert primes(1) == [2]
assert primes(5) == [2, 3, 5, 7, 11]
assert primes(0) == []

print("First 10 primes:", primes(10))


First 10 primes: [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]


In [30]:
# Step 1: first 64 primes
p64 = primes(64)

# Step 2: cube roots (float64 for precision)
roots = np.cbrt(np.array(p64, dtype=np.float64))

# Step 3: fractional part
frac = roots - np.floor(roots)

# Step 4: first 32 bits of fractional part
k_values = np.floor(frac * (2**32)).astype(np.uint32)

# Step 5: display as 8-hex-digit strings
k_hex = [f"{int(v):08x}" for v in k_values]

print("First 8 computed K constants:", k_hex[:8])
print("Last 8 computed K constants:", k_hex[-8:])
print("Total constants:", len(k_hex))


First 8 computed K constants: ['428a2f98', '71374491', 'b5c0fbcf', 'e9b5dba5', '3956c25b', '59f111f1', '923f82a4', 'ab1c5ed5']
Last 8 computed K constants: ['748f82ee', '78a5636f', '84c87814', '8cc70208', '90befffa', 'a4506ceb', 'bef9a3f7', 'c67178f2']
Total constants: 64


In [31]:
# Published SHA-256 K constants from NIST FIPS 180-4 (p.11)
K_FIPS = [
    "428a2f98","71374491","b5c0fbcf","e9b5dba5","3956c25b","59f111f1","923f82a4","ab1c5ed5",
    "d807aa98","12835b01","243185be","550c7dc3","72be5d74","80deb1fe","9bdc06a7","c19bf174",
    "e49b69c1","efbe4786","0fc19dc6","240ca1cc","2de92c6f","4a7484aa","5cb0a9dc","76f988da",
    "983e5152","a831c66d","b00327c8","bf597fc7","c6e00bf3","d5a79147","06ca6351","14292967",
    "27b70a85","2e1b2138","4d2c6dfc","53380d13","650a7354","766a0abb","81c2c92e","92722c85",
    "a2bfe8a1","a81a664b","c24b8b70","c76c51a3","d192e819","d6990624","f40e3585","106aa070",
    "19a4c116","1e376c08","2748774c","34b0bcb5","391c0cb3","4ed8aa4a","5b9cca4f","682e6ff3",
    "748f82ee","78a5636f","84c87814","8cc70208","90befffa","a4506ceb","bef9a3f7","c67178f2"
]

assert len(K_FIPS) == 64
assert k_hex == K_FIPS, "Computed constants do not match FIPS 180-4!"

print("✅ All 64 SHA-256 K constants match NIST FIPS 180-4 (p.11).")


✅ All 64 SHA-256 K constants match NIST FIPS 180-4 (p.11).


### Conclusion of Problem Two

The computed constants exactly match the 64 SHA-256 round constants published in
NIST FIPS 180-4 (page 11). This confirms that the procedure of extracting the first
32 bits of the fractional parts of the cube roots of the first 64 prime numbers has
been implemented correctly.

Deriving these constants programmatically demonstrates how seemingly arbitrary
values used in cryptographic algorithms are, in fact, deterministically generated
from well-defined mathematical properties. This helps prevent the use of
“hidden” or biased constants and contributes to the transparency of the SHA-256
design.


## Problem 3 — Padding (Block Parsing)

SHA-256 processes input messages in fixed-size **512-bit (64-byte) blocks**. Before
processing, messages must be padded according to the rules defined in
NIST FIPS 180-4 (§5.1.1 and §5.2.1).

The padding process consists of:

1. Appending a single `1` bit to the message (represented as the byte `0x80`).
2. Appending `0` bits until the message length (in bytes) is congruent to 56 modulo 64.
3. Appending the original message length as a **64-bit big-endian integer**, measured
   in bits.

This ensures that the padded message length is an exact multiple of 512 bits.
The generator function implemented below yields each padded 64-byte block in sequence.


In [32]:
def block_parse(msg: bytes):
    """
    Yield 512-bit (64-byte) SHA-256 message blocks from msg, including padding.

    Padding rules (NIST FIPS 180-4 §5.1.1 and §5.2.1):
    1) Append 0x80 (a single '1' bit followed by seven '0' bits)
    2) Append 0x00 bytes until the total length is 56 mod 64
    3) Append the original message length (in bits) as a 64-bit big-endian integer

    Parameters
    ----------
    msg : bytes
        The original message as a bytes-like object.

    Yields
    ------
    bytes
        The next 64-byte (512-bit) block of the padded message.
    """
    if not isinstance(msg, (bytes, bytearray)):
        raise TypeError("msg must be a bytes-like object (bytes or bytearray)")

    msg_len_bits = len(msg) * 8

    # 1) append 0x80
    padded = bytearray(msg)
    padded.append(0x80)

    # 2) pad with zeros until length ≡ 56 (mod 64)
    while (len(padded) % 64) != 56:
        padded.append(0x00)

    # 3) append length as 64-bit big-endian integer
    padded.extend(msg_len_bits.to_bytes(8, byteorder="big"))

    # yield 64-byte blocks
    for i in range(0, len(padded), 64):
        yield bytes(padded[i:i + 64])


In [33]:
# Smoke check: should always yield blocks of 64 bytes
for m in [b"", b"abc", b"a" * 55, b"a" * 56, b"a" * 64]:
    bs = list(block_parse(m))
    assert all(len(b) == 64 for b in bs)

print("✅ block_parse smoke check passed.")


✅ block_parse smoke check passed.


In [34]:
def hexdump_block(block: bytes, width: int = 16) -> str:
    """
    Format a 64-byte block as grouped hexadecimal bytes for notebook inspection.

    Parameters
    ----------
    block : bytes
        A 64-byte SHA-256 message block.
    width : int
        Number of bytes per line in the output.

    Returns
    -------
    str
        Multi-line formatted hex dump string.
    """
    if len(block) != 64:
        raise ValueError("Expected a 64-byte block")

    lines = []
    for i in range(0, 64, width):
        chunk = block[i:i + width]
        lines.append(" ".join(f"{b:02x}" for b in chunk))
    return "\n".join(lines)


In [35]:
print(hexdump_block(next(block_parse(b"abc"))))


61 62 63 80 00 00 00 00 00 00 00 00 00 00 00 00
00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 18


In [36]:
def collect_blocks(msg: bytes) -> list[bytes]:
    """Return all 64-byte blocks produced by block_parse(msg)."""
    return list(block_parse(msg))


# --- Test 1: Empty message ---
blocks_empty = collect_blocks(b"")
assert len(blocks_empty) == 1
assert blocks_empty[0][0] == 0x80
assert blocks_empty[0][-8:] == (0).to_bytes(8, "big")


# --- Test 2: Short message ("abc") ---
msg = b"abc"
blocks_abc = collect_blocks(msg)
assert len(blocks_abc) == 1
assert blocks_abc[0][:3] == msg
assert blocks_abc[0][3] == 0x80
assert blocks_abc[0][-8:] == (len(msg) * 8).to_bytes(8, "big")


# --- Test 3: 55-byte message (fits length in same block) ---
msg55 = b"a" * 55
blocks_55 = collect_blocks(msg55)
assert len(blocks_55) == 1
assert blocks_55[0][55] == 0x80
assert blocks_55[0][-8:] == (55 * 8).to_bytes(8, "big")


# --- Test 4: 56-byte message (requires extra block) ---
msg56 = b"a" * 56
blocks_56 = collect_blocks(msg56)
assert len(blocks_56) == 2
assert blocks_56[0][56] == 0x80
assert blocks_56[1][-8:] == (56 * 8).to_bytes(8, "big")


# --- Test 5: 64-byte message (exact block, then padding block) ---
msg64 = b"a" * 64
blocks_64 = collect_blocks(msg64)
assert len(blocks_64) == 2
assert blocks_64[0] == msg64
assert blocks_64[1][-8:] == (64 * 8).to_bytes(8, "big")


print("✅ block_parse padding tests passed.")


✅ block_parse padding tests passed.


In [37]:
demo_msg = b"abc"
demo_blocks = list(block_parse(demo_msg))

print(f"Message: {demo_msg!r}")
print(f"Number of 512-bit blocks produced: {len(demo_blocks)}\n")

print("Block 1 (hex):")
print(hexdump_block(demo_blocks[0]))

print("\nLast 8 bytes (bit-length field):", demo_blocks[0][-8:].hex())


Message: b'abc'
Number of 512-bit blocks produced: 1

Block 1 (hex):
61 62 63 80 00 00 00 00 00 00 00 00 00 00 00 00
00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 18

Last 8 bytes (bit-length field): 0000000000000018


### Conclusion of Problem Three

The `block_parse(msg)` generator implements SHA-256 preprocessing by producing 64-byte
blocks with correct padding and a final 64-bit big-endian length field. The tests
cover boundary conditions (55/56/64 bytes) that require either one or two output
blocks, providing confidence that the output matches the behaviour specified in
NIST FIPS 180-4.


## Problem 4 — Hashes (SHA-256 Compression Step)

In SHA-256, the compression function updates the current hash state using a 512-bit
message block (FIPS 180-4 §6.2.2). This involves:

- Parsing the 64-byte block into 16 big-endian 32-bit words
- Expanding these into a 64-word message schedule `W`
- Running 64 rounds using the SHA-256 functions (`Ch`, `Maj`, `Σ0`, `Σ1`) and constants `K`
- Adding the working variables back into the current hash state

The function implemented below computes the next hash state from a given `current`
state and `block`.


In [38]:
K = np.array(k_values, dtype=np.uint32)
assert K.shape == (64,)


In [39]:
def make_schedule_16(block: bytes) -> np.ndarray:
    """
    Parse a 64-byte SHA-256 message block into the first 16 message schedule words.
    """
    if len(block) != 64:
        raise ValueError("block must be exactly 64 bytes (512 bits)")

    W = np.zeros(64, dtype=np.uint32)

    for i in range(16):
        chunk = block[4*i:4*i+4]
        val = int.from_bytes(chunk, byteorder="big")
        W[i] = np.uint32(val)

    return W


# micro-tests
W0 = make_schedule_16(b"\x00" * 64)
assert W0.dtype == np.uint32 and W0.shape == (64,)
assert int(W0[0]) == 0 and int(W0[15]) == 0

W1 = make_schedule_16(b"\x00\x00\x00\x01" + b"\x00" * 60)
assert int(W1[0]) == 1

print("✅ make_schedule_16 micro-tests passed.")


✅ make_schedule_16 micro-tests passed.


In [40]:
def make_schedule(block: bytes) -> np.ndarray:
    """
    Build the full SHA-256 message schedule W[0..63] from a 64-byte block.
    """
    W = make_schedule_16(block)

    for t in range(16, 64):
        W[t] = np.uint32(
            sigma1(W[t - 2]) + W[t - 7] + sigma0(W[t - 15]) + W[t - 16]
        )

    return W


# sanity check
W0 = make_schedule(b"\x00" * 64)
assert W0.shape == (64,) and W0.dtype == np.uint32
print("✅ make_schedule sanity check passed.")


✅ make_schedule sanity check passed.


In [41]:
def hash(current: np.ndarray, block: bytes) -> np.ndarray:
    """
    Perform one SHA-256 compression step.

    Parameters
    ----------
    current : np.ndarray
        Current hash state (8 uint32 values).
    block : bytes
        512-bit message block (64 bytes).

    Returns
    -------
    np.ndarray
        Updated hash state (8 uint32 values).
    """
    if current.shape != (8,):
        raise ValueError("current must contain 8 uint32 values")

    W = make_schedule(block)

    a, b, c, d, e, f, g, h = current

    for t in range(64):
        T1 = np.uint32(h + Sigma1(e) + Ch(e, f, g) + K[t] + W[t])
        T2 = np.uint32(Sigma0(a) + Maj(a, b, c))

        h = g
        g = f
        f = e
        e = np.uint32(d + T1)
        d = c
        c = b
        b = a
        a = np.uint32(T1 + T2)

    return np.array([
        np.uint32(current[0] + a),
        np.uint32(current[1] + b),
        np.uint32(current[2] + c),
        np.uint32(current[3] + d),
        np.uint32(current[4] + e),
        np.uint32(current[5] + f),
        np.uint32(current[6] + g),
        np.uint32(current[7] + h),
    ], dtype=np.uint32)


In [42]:
import hashlib

# SHA-256 initial hash values (FIPS 180-4 §5.3.3)
H0 = np.array([
    0x6a09e667,
    0xbb67ae85,
    0x3c6ef372,
    0xa54ff53a,
    0x510e527f,
    0x9b05688c,
    0x1f83d9ab,
    0x5be0cd19
], dtype=np.uint32)


def sha256_digest(msg: bytes) -> bytes:
    """
    Compute SHA-256 digest using block_parse(msg) + hash(state, block).
    Returns the raw 32-byte digest.
    """
    state = H0.copy()
    for block in block_parse(msg):
        state = hash(state, block)

    return b"".join(int(x).to_bytes(4, "big") for x in state)


def sha256_hex(msg: bytes) -> str:
    return sha256_digest(msg).hex()


# Known test messages
tests = [
    b"",
    b"abc",
    b"hello",
    b"The quick brown fox jumps over the lazy dog",
]

for m in tests:
    ours = sha256_hex(m)
    ref = hashlib.sha256(m).hexdigest()
    print(f"Message: {m!r}")
    print(" ours:", ours)
    print(" ref :", ref)
    print()
    assert ours == ref

print("✅ SHA-256 verified against hashlib test vectors.")


Message: b''
 ours: e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855
 ref : e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855

Message: b'abc'
 ours: ba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad
 ref : ba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad

Message: b'hello'
 ours: 2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824
 ref : 2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824

Message: b'The quick brown fox jumps over the lazy dog'
 ours: d7a8fbb307d7809469ca9abcb0082e4f8d5651e46d3cdb762d02d0bf37c9e592
 ref : d7a8fbb307d7809469ca9abcb0082e4f8d5651e46d3cdb762d02d0bf37c9e592

✅ SHA-256 verified against hashlib test vectors.


/tmp/ipykernel_2051/18906757.py:9: RuntimeWarning: overflow encountered in scalar add
  sigma1(W[t - 2]) + W[t - 7] + sigma0(W[t - 15]) + W[t - 16]
/tmp/ipykernel_2051/1110595970.py:25: RuntimeWarning: overflow encountered in scalar add
  T1 = np.uint32(h + Sigma1(e) + Ch(e, f, g) + K[t] + W[t])
/tmp/ipykernel_2051/1110595970.py:26: RuntimeWarning: overflow encountered in scalar add
  T2 = np.uint32(Sigma0(a) + Maj(a, b, c))
/tmp/ipykernel_2051/1110595970.py:31: RuntimeWarning: overflow encountered in scalar add
  e = np.uint32(d + T1)
/tmp/ipykernel_2051/1110595970.py:35: RuntimeWarning: overflow encountered in scalar add
  a = np.uint32(T1 + T2)
/tmp/ipykernel_2051/1110595970.py:39: RuntimeWarning: overflow encountered in scalar add
  np.uint32(current[1] + b),
/tmp/ipykernel_2051/1110595970.py:41: RuntimeWarning: overflow encountered in scalar add
  np.uint32(current[3] + d),
/tmp/ipykernel_2051/1110595970.py:42: RuntimeWarning: overflow encountered in scalar add
  np.uint32(current

### Conclusion of Problem Four

The SHA-256 compression function was implemented following NIST FIPS 180-4 §6.2.2.
Using the padding/block generator from Problem 3, the implementation reproduces
the same digests as Python’s `hashlib.sha256` for multiple test vectors, providing
confidence that the round function, message schedule, and 32-bit arithmetic are correct.


## Problem 5 — Passwords (Dictionary Attack)

The three values provided are SHA-256 digests of common passwords. Because they were:

- hashed only once (fast),
- with no salt (same password → same hash),
- and are described as "common passwords",

they can be recovered efficiently by performing a **dictionary attack**: hash many
candidate passwords and compare against the target hashes.

In this section I:
1. Encode candidate passwords as UTF-8
2. Compute SHA-256 digests
3. Match digests against the given hashes
4. Explain why this attack is effective and how to improve password hashing


In [43]:
import hashlib
from typing import Iterable

TARGET_HASHES = [
    "5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8",
    "873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34",
    "b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342",
]

TARGET_SET = set(h.lower() for h in TARGET_HASHES)


def sha256_hex(s: str) -> str:
    """Return SHA-256 hex digest of a UTF-8 encoded string."""
    return hashlib.sha256(s.encode("utf-8")).hexdigest()


def find_matches(candidates: Iterable[str]) -> dict[str, str]:
    """
    Return a mapping of hash -> recovered password for any candidates that match
    the target hashes.
    """
    found: dict[str, str] = {}
    for pw in candidates:
        pw = pw.strip("\n\r")
        if not pw:
            continue
        h = sha256_hex(pw)
        if h in TARGET_SET and h not in found:
            found[h] = pw
    return found


In [44]:
common_candidates = [
    "password", "123456", "123456789", "qwerty", "abc123", "letmein",
    "admin", "welcome", "iloveyou", "monkey", "dragon", "football",
    "password1", "123123", "111111", "000000", "secret", "passw0rd",
]

found = find_matches(common_candidates)

print("Matches found:", len(found))
for h, pw in found.items():
    print(h, "->", pw)


Matches found: 1
5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8 -> password


In [45]:
from pathlib import Path
from typing import Iterable

def wordlist_candidates(path: str) -> Iterable[str]:
    """
    Yield candidate passwords from a newline-separated wordlist file.
    """
    p = Path(path)
    with p.open("r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            candidate = line.strip()
            if candidate:
                yield candidate


wordlist_path = "../wordlists/top-passwords.txt"

if Path(wordlist_path).exists():
    found2 = find_matches(wordlist_candidates(wordlist_path))
    print("Matches found from wordlist:", len(found2))
    for h, pw in found2.items():
        print(h, "->", pw)
else:
    print(f"Wordlist not found at: {wordlist_path}")


Matches found from wordlist: 1
5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8 -> password


In [46]:
from typing import Iterable

def generate_variants(words: Iterable[str]) -> Iterable[str]:
    """
    Generate common password variants from base words.

    Variants include:
    - case transformations (lower/upper/capitalized)
    - common suffixes (digits, punctuation)
    - common years
    """
    suffixes = [
        "", "!", ".", "@", "#",
        "1", "11", "12", "123", "1234", "12345",
        "01", "007", "69",
        "2020", "2021", "2022", "2023", "2024", "2025",
    ]

    for w in words:
        w = w.strip()
        if not w:
            continue

        bases = {w, w.lower(), w.upper(), w.capitalize()}

        for b in bases:
            for s in suffixes:
                yield b + s


# Run variant-based attack
base_words = list(wordlist_candidates(wordlist_path))
found_variants = find_matches(generate_variants(base_words))

print("Matches found from variants:", len(found_variants))
for h, pw in found_variants.items():
    print(h, "->", pw)


Matches found from variants: 1
5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8 -> password
